#### 1. committed space 和 virtual space
1. 虚拟机进程在初始化时, 会保留( reserve )整个堆大小的内存  
2. 虚拟机的保留内存 (reserve space) 通过 `Xmx` 指定;   
   如果 `Xms` 的配置比 `Xmx` 的配置要小, 则并非所有的保留空间 (reserved space) 立即提交给 jvm, 未提交的空间在下图被标记为 `virtual` 的 (提交: `commit`)
   <img src="img/jsgct_dt_006_prm_gn_sz.png"/>
   
   
#### 2. Total heap
1. 如下的堆内存伸缩理论, 不适用于`Parallel GC`; 不过控制堆的 total size 和 各分代的大小仍适用于`Parallel GC` 
2. 默认情况下, 整个堆的大小会扩张和收缩, 来保证一定比例的空白空间存储存活对象. 这个比例由2个参数控制:  

|  Parameter   | Default Value  |
|  ----  | ----  |
|MinHeapFreeRatio|40|
|MaxHeapFreeRatio|70|

    (1) 这些参数表示, 如果某个代的空闲空间小于40%, 则该代会扩张直到空闲大小占到40%;   
    (2) 如果某个代的空闲空间大于70%, 则该代会收缩, 直到空闲空间占到该代的70%


#### 3. The Young Generation
除了 Total heap ,另一个 GC 表现的就是 Young Generation 的大小:   
1. Young Generation的影响   
    Young Generation 越大, 发生 minor gc 的次数就越少; 但是相反, 对于一个限制了 total size 的 jvm 进程, 更大的 Young Generation , 意味着 tenured generation 更小, 则 major gc 发生的会更频繁. 最优的选择是根据应用程序的对象的生命周期(`lifetime`)  
2. Young Generation的比例由 `NewRatio` 配置   
    比如, 设置 `-XX:NewRatio=3 ` , 意味着 Young Generation 和 tenured generation 的比例为 1:3 ; 即: eden 和 survivor 空间共占 Total heap 的 $\frac{1}{4}$  
3. `NewSize` 和 `MaxNewSize` 限制了 young generation 大小的下限和上限. 将这两个值设置成相同的可以固定住 young generation 的大小, 就像把 `-Xms` 和 `-Xmx` 设置成相同值可以固定 Total heap size 一样. 这两个参数可以比 `NewRatio` 更细粒度的调试 young generation .参数默认值: 

|Parameter	|Server JVM Default Value|
| ------ | ------ |
|NewRatio| 2
|NewSize |1310M|
|MaxNewSize | not limited|  

    Young Generation 的最大大小是可以由 total heap 的大小和参数 `NewRatio` 计算出来的. `MaxNewSize` 的默认值是 "not limited" 表示计算结果不受限制, 除非明确指定了 `MaxNewSize` 的大小

#### 4. Survivor Space Sizing (次要)
1. 参数 `SurvivorRatio` 用来调试 Survivor Space , 不过这通常并不怎么影响性能   
   比如 `-XX:SurvivorRatio=6` 设置每个 Survivor Space 和 eden space 的比例为 1:6 , 即每个占 young generation 的$\frac{1}{8}$ (因为有2个 Survivor Space). 
   
2. Survivor Space 影响    
   如果 Survivor Space 设置的太小, 会导致复制集合溢出, 直接拷贝到 tenured generation ; 如果设置的太大, 会导致有很多无法使用的空区域. jvm 在每次进行 gc 时, 都会选择一个阈值: 表示对象进入 tenured generation 前可被拷贝的次数; 这个阈值的选择标准是: 确保 Survivor Space 内的对象只占自己总容量的一半; 参数的默认值如下: 
   
|Parameter	|Server JVM Default Value|
| ------ | ------ |
|SurvivorRatio|  8|

#### 5. 服务端程序的一般性建议: 
1. 首先确定打算给这个jvm进程的最大 heap size ; 然后针对 young generation 的大小绘制一个性能表现图, 从中找出最优的 young generation 大小     
```
   [注] 最大的 heap size 应该小于物理机的总共大小; 否则会引起代价昂贵的页缺失和抖动(trashing)问题
```
2. 一旦总共的 heap size 确定下来, 然后就该逐步加大 young generation 的大小 ( tenured generation 在过程中的 size 逐步减小). 当然要保证 tenured generation 足够大来保持所有应用使用的存活对象, 还要有额外 10% ~ 20% 的松弛空间 (slace space)     
```   
   [注] 由于 tenured generation 的限制, 应该给 young generation 足够的空间. 且随着 cpu 核数的增加, young generation 的空间也应该同步增加, 因为应用可以并行的跑
```